In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pcb-defects/PCB_DATASET/rotate.py
/kaggle/input/pcb-defects/PCB_DATASET/PCB_USED/10.JPG
/kaggle/input/pcb-defects/PCB_DATASET/PCB_USED/01.JPG
/kaggle/input/pcb-defects/PCB_DATASET/PCB_USED/09.JPG
/kaggle/input/pcb-defects/PCB_DATASET/PCB_USED/05.JPG
/kaggle/input/pcb-defects/PCB_DATASET/PCB_USED/06.JPG
/kaggle/input/pcb-defects/PCB_DATASET/PCB_USED/04.JPG
/kaggle/input/pcb-defects/PCB_DATASET/PCB_USED/12.JPG
/kaggle/input/pcb-defects/PCB_DATASET/PCB_USED/11.JPG
/kaggle/input/pcb-defects/PCB_DATASET/PCB_USED/08.JPG
/kaggle/input/pcb-defects/PCB_DATASET/PCB_USED/07.JPG
/kaggle/input/pcb-defects/PCB_DATASET/Annotations/Mouse_bite/01_mouse_bite_11.xml
/kaggle/input/pcb-defects/PCB_DATASET/Annotations/Mouse_bite/01_mouse_bite_17.xml
/kaggle/input/pcb-defects/PCB_DATASET/Annotations/Mouse_bite/08_mouse_bite_08.xml
/kaggle/input/pcb-defects/PCB_DATASET/Annotations/Mouse_bite/08_mouse_bite_02.xml
/kaggle/input/pcb-defects/PCB_DATASET/Annotations/Mouse_bite/07_mouse_bite_03.xml
/k

In [8]:
import os
import shutil
import random
import xml.etree.ElementTree as ET
from tqdm import tqdm

# ======================================================
# CONFIG
# ======================================================
DATA_ROOT = "/kaggle/input/pcb-defects/PCB_DATASET"
IMG_ROOT  = os.path.join(DATA_ROOT, "images")
ANN_ROOT  = os.path.join(DATA_ROOT, "Annotations")
OUT_ROOT  = "/kaggle/working/dataset"

# We normalize these for comparison (lowercase, no spaces, no underscores)
CLASS_NAMES = ["mouse_bite", "spur", "open_circuit", "short", "missing_hole", "spurious_copper"]

def normalize_label(name):
    """Turns 'Missing_Hole' or 'missing hole' into 'missinghole'"""
    return name.lower().replace(" ", "").replace("_", "").strip()

# Map the normalized strings to IDs
NORM_CLASS_TO_ID = {normalize_label(c): i for i, c in enumerate(CLASS_NAMES)}

# ======================================================
# THE CONVERSION FUNCTION
# ======================================================
def convert_xml(xml_path, txt_path):
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        size = root.find("size")
        W = int(size.find("width").text)
        H = int(size.find("height").text)
        
        lines = []
        for obj in root.findall("object"):
            raw_name = obj.find("name").text
            clean_name = normalize_label(raw_name)

            if clean_name not in NORM_CLASS_TO_ID:
                # Log this so you can see exactly what the XML contains
                print(f"⚠️ Warning: Found '{raw_name}' in XML, but it doesn't match your list!")
                continue

            cls_id = NORM_CLASS_TO_ID[clean_name]
            box = obj.find("bndbox")
            
            xmin = float(box.find("xmin").text)
            ymin = float(box.find("ymin").text)
            xmax = float(box.find("xmax").text)
            ymax = float(box.find("ymax").text)

            # YOLO Normalization
            x = ((xmin + xmax) / 2) / W
            y = ((ymin + ymax) / 2) / H
            w = (xmax - xmin) / W
            h = (ymax - ymin) / H
            
            lines.append(f"{cls_id} {x:.6f} {y:.6f} {w:.6f} {h:.6f}")

        if lines:
            with open(txt_path, "w") as f:
                f.write("\n".join(lines))
            return True
        return False
    except Exception as e:
        return False

# ======================================================
# DATA COLLECTION & SPLITTING
# ======================================================
samples = []
for folder in os.listdir(IMG_ROOT):
    img_dir = os.path.join(IMG_ROOT, folder)
    ann_dir = os.path.join(ANN_ROOT, folder)
    if not os.path.isdir(img_dir): continue

    for img_name in os.listdir(img_dir):
        if not img_name.lower().endswith((".jpg", ".png", ".jpeg")): continue
        
        xml_name = os.path.splitext(img_name)[0] + ".xml"
        xml_path = os.path.join(ann_dir, xml_name)
        
        if os.path.exists(xml_path):
            samples.append((folder, img_name, xml_path))

random.seed(42)
random.shuffle(samples)

# Splits: 70/20/10
train_end = int(len(samples) * 0.7)
val_end = int(len(samples) * 0.9)

splits = {
    "train": samples[:train_end],
    "val":   samples[train_end:val_end],
    "test":  samples[val_end:]
}

# ======================================================
# EXECUTION
# ======================================================
for split, items in splits.items():
    os.makedirs(f"{OUT_ROOT}/{split}/images", exist_ok=True)
    os.makedirs(f"{OUT_ROOT}/{split}/labels", exist_ok=True)
    
    for folder, img_name, xml_path in tqdm(items, desc=f"Writing {split}"):
        new_base = f"{folder}_{os.path.splitext(img_name)[0]}"
        lbl_dst = f"{OUT_ROOT}/{split}/labels/{new_base}.txt"
        img_dst = f"{OUT_ROOT}/{split}/images/{folder}_{img_name}"
        
        # Only copy image if we successfully created a non-empty label file
        if convert_xml(xml_path, lbl_dst):
            shutil.copy(os.path.join(IMG_ROOT, folder, img_name), img_dst)

print("🚀 Done! Check /kaggle/working/dataset/train/labels/ to see the results.")

Writing test: 100%|██████████| 70/70 [00:00<00:00, 390.57it/s]

🚀 Done! Check /kaggle/working/dataset/train/labels/ to see the results.


In [9]:
with open("/kaggle/working/dataset/data.yaml", "w") as f:
    f.write(
        "path: /kaggle/working/dataset\n"
        "train: train/images\n"
        "val: val/images\n"
        "test: test/images\n\n"
        f"nc: {len(CLASS_NAMES)}\n"
        f"names: {CLASS_NAMES}\n"
    )


In [ ]:
!cat /kaggle/working/dataset.yaml


In [ ]:
import torch
from ultralytics import YOLO


dataset_yaml = """
path: /kaggle/working/dataset

train: train
val: val
test: test

nc: 6

names:
  - mouse_bite
  - spur
  - open_circuit
  - short
  - missing_hole
  - spurious_copper
"""


with open("dataset.yaml", "w") as f:
    f.write(dataset_yaml)

model = YOLO("yolov8n.pt")  # or yolov8s.pt / yolov8m.pt
model = model.to("cuda")
model.train( data="dataset.yaml", epochs=100, imgsz=640, batch=16, iou=0.5, mosaic=0.5, mixup=0.0, erasing=0.0, auto_augment=None, device=0 )


In [ ]:
results = model(
   "/kaggle/working/dataset/train/images/Spurious_copper_05_spurious_copper_01.jpg",

)


In [ ]:
# res = results[0]

# print("Boxes:", res.boxes)
# print("Classes:", res.boxes.cls)
# print("Confidences:", res.boxes.conf)

res = results[0]

img_with_boxes = res.plot()   # numpy array (BGR)
import matplotlib.pyplot as plt

plt.imshow(img_with_boxes)
plt.axis("off")



In [ ]:
# import os
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# from ultralytics import YOLO

# # -------------------------------------------------
# # 1️⃣ Define evaluation function
# # -------------------------------------------------
# import numpy as np
# import matplotlib.pyplot as plt

# def yolo_evaluation(yolo_results, class_names=None, iou_threshold=0.5, conf_threshold=0.25):
#     """
#     Analyze YOLO predictions and compute evaluation metrics: TP, FP, FN, Precision, Recall, F1.
    
#     Args:
#         yolo_results: List of YOLO Result objects (results[i] = image i)
#         class_names: List of class names (model.names)
#         iou_threshold: IoU threshold to consider a detection as True Positive
#         conf_threshold: Confidence threshold to filter predictions

#     Returns:
#         metrics_dict: Dictionary with precision, recall, F1, per-class stats
#     """
    
#     # -----------------------------
#     # Helper: compute IoU
#     # -----------------------------
#     def compute_iou(box1, box2):
#         x1 = max(box1[0], box2[0])
#         y1 = max(box1[1], box2[1])
#         x2 = min(box1[2], box2[2])
#         y2 = min(box1[3], box2[3])
#         inter_area = max(0, x2 - x1) * max(0, y2 - y1)
#         box1_area = (box1[2]-box1[0])*(box1[3]-box1[1])
#         box2_area = (box2[2]-box2[0])*(box2[3]-box2[1])
#         union_area = box1_area + box2_area - inter_area
#         return inter_area / union_area if union_area > 0 else 0

#     # -----------------------------
#     # Initialize per-class stats
#     # -----------------------------
#     n_classes = len(class_names) if class_names else 1
#     TP = np.zeros(n_classes)
#     FP = np.zeros(n_classes)
#     FN = np.zeros(n_classes)
    
#     # -----------------------------
#     # Process each image
#     # -----------------------------
#     for res in yolo_results:
#         # Predicted boxes
#         pred_boxes = res.boxes.xyxy.cpu().numpy()
#         pred_classes = res.boxes.cls.cpu().numpy().astype(int)
#         pred_scores = res.boxes.conf.cpu().numpy()
        
#         # Filter low confidence
#         mask = pred_scores >= conf_threshold
#         pred_boxes = pred_boxes[mask]
#         pred_classes = pred_classes[mask]
        
#         # Ground truth boxes: [x1, y1, x2, y2, class_id]
#         gt_boxes = res.boxes.data.cpu().numpy() if hasattr(res.boxes, "data") else None
#         if gt_boxes is None or len(gt_boxes) == 0:
#             continue
        
#         gt_boxes_list = gt_boxes.tolist()
#         detected_gt = [False] * len(gt_boxes_list)
        
#         for pb, pc in zip(pred_boxes, pred_classes):
#             ious = [compute_iou(pb, gt) if pc == int(gt[4]) else 0 for gt in gt_boxes_list]
#             if ious and max(ious) >= iou_threshold:
#                 gt_idx = np.argmax(ious)
#                 if not detected_gt[gt_idx]:
#                     TP[pc] += 1
#                     detected_gt[gt_idx] = True
#                 else:
#                     FP[pc] += 1
#             else:
#                 FP[pc] += 1
        
#         # Count FNs
#         for idx, gt in enumerate(gt_boxes_list):
#             if not detected_gt[idx]:
#                 FN[int(gt[4])] += 1
    
#     # -----------------------------
#     # Compute metrics
#     # -----------------------------
#     precision = TP / (TP + FP + 1e-8)
#     recall = TP / (TP + FN + 1e-8)
#     f1 = 2 * precision * recall / (precision + recall + 1e-8)
    
#     metrics_dict = {
#         "TP": TP,
#         "FP": FP,
#         "FN": FN,
#         "Precision": precision,
#         "Recall": recall,
#         "F1": f1,
#         "mPrecision": np.mean(precision),
#         "mRecall": np.mean(recall),
#         "mF1": np.mean(f1)
#     }
    
#     # -----------------------------
#     # Print per-class metrics
#     # -----------------------------
#     if class_names:
#         print(f"{'Class':15} {'TP':>5} {'FP':>5} {'FN':>5} {'Prec':>6} {'Recall':>6} {'F1':>6}")
#         for i, cname in enumerate(class_names):
#             print(f"{cname:15} {int(TP[i]):5d} {int(FP[i]):5d} {int(FN[i]):5d} "
#                   f"{precision[i]:6.2f} {recall[i]:6.2f} {f1[i]:6.2f}")
    
#     # -----------------------------
#     # Simple bar plot per class
#     # -----------------------------
#     if class_names:
#         x = np.arange(len(class_names))
#         width = 0.25
#         plt.figure(figsize=(10,5))
#         plt.bar(x - width, precision, width=width, label="Precision")
#         plt.bar(x, recall, width=width, label="Recall")
#         plt.bar(x + width, f1, width=width, label="F1")
#         plt.xticks(x, class_names, rotation=45)
#         plt.ylabel("Score")
#         plt.ylim(0, 1)
#         plt.title("YOLO Evaluation Metrics per Class")
#         plt.legend()
#         plt.show()
    
#     return metrics_dict


# # -------------------------------------------------
# # 2️⃣ Inference + Evaluation Pipeline
# # -------------------------------------------------
# def run_yolo_test_eval(model_path, test_folder, conf_threshold=0.4, iou_threshold=0.5, visualize=True):
#     """
#     Run YOLO inference on a folder of test images and evaluate.
    
#     Args:
#         model_path: path to trained YOLO weights (.pt)
#         test_folder: folder with test images
#         conf_threshold: confidence threshold
#         iou_threshold: IoU threshold for TP/FP
#         visualize: whether to show images
#     """
#     # Load model
#     model = YOLO(model_path)
    
#     # Get test images
#     test_images = [os.path.join(test_folder, f) 
#                    for f in os.listdir(test_folder) 
#                    if f.lower().endswith((".jpg",".png",".jpeg"))]
    
#     # Run inference
#     results = []
#     for img_path in test_images:
#         res = model.predict(source=img_path, conf=conf_threshold, imgsz=640, verbose=False)
#         results.append(res[0])
    
#     # Evaluate
#     metrics = yolo_evaluation(results, class_names=model.names, iou_threshold=iou_threshold, conf_threshold=conf_threshold)
    
#     print("\n=== Overall Metrics ===")
#     print(f"Mean Precision: {metrics['mPrecision']:.3f}")
#     print(f"Mean Recall   : {metrics['mRecall']:.3f}")
#     print(f"Mean F1       : {metrics['mF1']:.3f}")
    
#     # Visualization
#     if visualize:
#         for res in results[:5]:  # first 5 images
#             img = res.orig_img.copy()
#             boxes = res.boxes.xyxy.cpu().numpy()
#             classes = res.boxes.cls.cpu().numpy().astype(int)
#             scores = res.boxes.conf.cpu().numpy()
            
#             for box, cls, conf in zip(boxes, classes, scores):
#                 if conf < conf_threshold:
#                     continue
#                 x1, y1, x2, y2 = map(int, box)
#                 label = f"{model.names[int(cls)]}: {conf:.2f}"
#                 cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
#                 cv2.putText(img, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
            
#             plt.figure(figsize=(8,8))
#             plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#             plt.axis("off")
#             plt.show()
    
#     return metrics

# # -------------------------------------------------
# # 3️⃣ Example usage
# # -------------------------------------------------
# if __name__ == "__main__":
#     metrics = run_yolo_test_eval(
#         model_path="/kaggle/working/model.pt",             # replace with your trained weights
#         test_folder="/kaggle/working/dataset/test/images",
#         conf_threshold=0.4,
#         iou_threshold=0.5,
#         visualize=True
#     )


In [26]:
import os
import numpy as np
from collections import defaultdict

# -----------------------------
# IoU
# -----------------------------
def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2]-box1[0])*(box1[3]-box1[1])
    area2 = (box2[2]-box2[0])*(box2[3]-box2[1])
    union = area1 + area2 - inter
    return inter / union if union > 0 else 0

# -----------------------------
# YOLO label → pixel box
# -----------------------------
def yolo_to_xyxy(xc, yc, w, h, H, W):
    x1 = (xc - w/2) * W
    y1 = (yc - h/2) * H
    x2 = (xc + w/2) * W
    y2 = (yc + h/2) * H
    return [x1, y1, x2, y2]

# -----------------------------
# Main evaluation function
# -----------------------------
def evaluate_detection(results, label_dir, class_names,
                       iou_thr=0.5, conf_thr=0.25):

    stats = defaultdict(lambda: {"TP":0, "FP":0, "FN":0, "IoU":[]})

    for res in results:
        H, W = res.orig_img.shape[:2]
        label_path = os.path.join(
            label_dir, os.path.basename(res.path).rsplit(".",1)[0] + ".txt"
        )

        # ---------- Load GT ----------
        gt_boxes = []
        with open(label_path) as f:
            for line in f:
                c, xc, yc, w, h = map(float, line.split())
                gt_boxes.append([*yolo_to_xyxy(xc, yc, w, h, H, W), int(c)])

        matched_gt = [False]*len(gt_boxes)

        # ---------- Predictions ----------
        boxes = res.boxes.xyxy.cpu().numpy()
        classes = res.boxes.cls.cpu().numpy().astype(int)
        confs = res.boxes.conf.cpu().numpy()

        mask = confs >= conf_thr
        boxes, classes = boxes[mask], classes[mask]

        # ---------- Match predictions ----------
        for pb, pc in zip(boxes, classes):
            best_iou, best_idx = 0, -1
            for i, gt in enumerate(gt_boxes):
                if pc == gt[4]:
                    iou = compute_iou(pb, gt[:4])
                    if iou > best_iou:
                        best_iou, best_idx = iou, i

            if best_iou >= iou_thr and best_idx != -1:
                if not matched_gt[best_idx]:
                    stats[pc]["TP"] += 1
                    stats[pc]["IoU"].append(best_iou)
                    matched_gt[best_idx] = True
                else:
                    stats[pc]["FP"] += 1
            else:
                stats[pc]["FP"] += 1

        # ---------- FN ----------
        for i, gt in enumerate(gt_boxes):
            if not matched_gt[i]:
                stats[gt[4]]["FN"] += 1

    # -----------------------------
    # Print table
    # -----------------------------
    print("\n=== Detection Metrics ===")
    print(f"{'Class':18s} {'TP':>4} {'FP':>4} {'FN':>4} {'Prec':>6} {'Rec':>6} {'F1':>6} {'mIoU':>6}")

    P_all, R_all, F1_all = [], [], []

    for cid, cname in enumerate(class_names):
        TP = stats[cid]["TP"]
        FP = stats[cid]["FP"]
        FN = stats[cid]["FN"]

        prec = TP / (TP + FP + 1e-9)
        rec  = TP / (TP + FN + 1e-9)
        f1   = 2*prec*rec / (prec + rec + 1e-9)
        miou = np.mean(stats[cid]["IoU"]) if stats[cid]["IoU"] else 0

        print(f"{cname:18s} {TP:4d} {FP:4d} {FN:4d} {prec:6.2f} {rec:6.2f} {f1:6.2f} {miou:6.2f}")

        P_all.append(prec)
        R_all.append(rec)
        F1_all.append(f1)

    print("\n=== Overall ===")
    print(f"Mean Precision: {np.mean(P_all):.3f}")
    print(f"Mean Recall   : {np.mean(R_all):.3f}")
    print(f"Mean F1       : {np.mean(F1_all):.3f}")


results = []
for img in os.listdir("/kaggle/working/dataset/test/images"):
    res = model(os.path.join(test_images, img))[0]
    results.append(res)

evaluate_detection(
    results,
    label_dir="/kaggle/working/dataset/test/labels",
    class_names=list(model.names.values()),
    iou_thr=0.5,
    conf_thr=0.25
)



image 1/1 /kaggle/working/dataset/test/images/Missing_hole_07_missing_hole_03.jpg: 448x640 5 missing_holes, 1 spurious_copper, 7.0ms
Speed: 2.6ms preprocess, 7.0ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /kaggle/working/dataset/test/images/Open_circuit_07_open_circuit_10.jpg: 448x640 4 open_circuits, 1 spurious_copper, 5.8ms
Speed: 2.4ms preprocess, 5.8ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /kaggle/working/dataset/test/images/Open_circuit_08_open_circuit_09.jpg: 512x640 1 mouse_bite, 4 open_circuits, 6.7ms
Speed: 2.9ms preprocess, 6.7ms inference, 1.1ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /kaggle/working/dataset/test/images/Open_circuit_08_open_circuit_07.jpg: 512x640 4 open_circuits, 6.1ms
Speed: 2.8ms preprocess, 6.1ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /kaggle/working/dataset/test/images/Mouse_bite_04_mouse_bite_01.jpg: 544x640 2 mouse_bites, 7